In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Deliverable2").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

In [4]:
# Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count equalt to or greater than 20
vote_over_20 = df.filter("total_votes >= 20")
vote_over_20.show()
vote_over_20.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

41458

In [5]:
# Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_votes = vote_over_20.filter(vote_over_20["helpful_votes"]/ vote_over_20["total_votes"] >= 0.5)
helpful_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

In [6]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_program = helpful_votes.filter(helpful_votes["vine"] == 'Y')
vine_program.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53093730|R29V4UDSS053D8|B00P6ORMR4|     406381545|Aquasana AQ-RO-3....|Home Improvement|          3|           33|         38|   Y|                N|Not for average h...|This filter is ov...| 2015-08-30|
|         US|   25984637|R1I2D068WC37PA|B00ZWBURWI|     302288140|Myfox Security Ca...|Home Improvement|          3|    

In [7]:
# Repeat previous step, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
vine_program_unpaid = helpful_votes.filter(helpful_votes["vine"] == 'N')
vine_program_unpaid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...| 2015-08-31|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|Home Improvement|          5|    

In [10]:
# Total Number of reviews, which are paid
display("Total Number of reviews in the Vine Program Paid: %s" % vine_program.count())
# The number of 5-star reviews
display("The number of 5-star in the Vine Program Paid: %s" % vine_program.filter('star_rating == 5').count())
# The percentage of 5-start reviews
display("The percentage of 5-star reviews in the Vine Program Paid: %s" % "{:.2%}".format(vine_program.filter("star_rating == 5").count()/vine_program.count()))

'Total Number of reviews in the Vine Program Paid: 266'

'The number of 5-star in the Vine Program Paid: 125'

'The percentage of 5-star reviews in the Vine Program Paid: 46.99%'

In [11]:
# Total Number of reviews, which are unpaid
display("Total Number of reviews in the Vine Program Unpaid: %s" % vine_program_unpaid.count())
# The number of 5-star reviews
display("The number of 5-star in the Vine Program Unpaid: %s" % vine_program_unpaid.filter('star_rating == 5').count())
# The percentage of 5-start reviews
display("The percentage of 5-star reviews in the Vine Program Unpaid: %s" % "{:.2%}".format(vine_program_unpaid.filter("star_rating == 5").count()/vine_program_unpaid.count()))

'Total Number of reviews in the Vine Program Unpaid: 38829'

'The number of 5-star in the Vine Program Unpaid: 18246'

'The percentage of 5-star reviews in the Vine Program Unpaid: 46.99%'